In [1]:
import pandas as pd
import numpy as np
import json
from scipy.spatial.transform import Rotation as R

In [2]:
input_curves = pd.read_table('curves.json',header=None).iloc[0,0]

In [3]:
jdata = json.loads(input_curves)

In [4]:
len(jdata)

236

In [5]:
list_camera = list()
for i in range(len(jdata)):
    x_i = jdata[i]
    dict_camera_i = x_i.copy()
    if(i==len(jdata)-1):
        x_i_next = jdata[i]
    else:
        x_i_next = jdata[i+1]
    if(x_i_next['branch_id'] != x_i['branch_id']):
        ##for now when it comes to the end point or branching point, i'm using the point itself as the next point  
        ##we can modify it later in JS.
        x_i_next = jdata[i]
    
    vec = np.array([x_i_next['x'] - x_i['x'],x_i_next['y'] - x_i['y'],x_i_next['z'] - x_i['z']])
    dict_camera_i['rotation'] = [np.arccos(vec[0]),np.arccos(vec[1]),np.arccos(vec[2])]
    list_camera.append(dict_camera_i)

In [6]:
len(list_camera)

236